In [ ]:
%reload_ext autoreload
%autoreload 2

from peptdeep.pretrained_models import ModelManager
from peptdeep import settings
from alphabase.psm_reader import psm_reader_provider

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import sklearn.metrics as sk
import sklearn.linear_model as sk_lm
import pickle

%run alpha_pept_deep_methods.ipynb

In [ ]:
# load df in max_quant format
df = pd.read_csv("evidence_freshfrozen_base.txt",  sep="\t")

In [ ]:
# load df with comparison of prediction to experimental values
df_comp = pd.read_csv('prediction.txt', sep = '\t')

# Linear Regression CCS

### Train

In [4]:
# Choose category for which Linear Regression should be performed
cat = 'CCS' # '1/K0'
if cat == 'CCS':
    pred = 'ccs_pred'
else:
    pred = 'IM_pred'
n = 10000

In [5]:
# Sample Training Set
df_train = df_comp.sample(n=n,axis=0, random_state=42)
%run alpha_pept_deep_methods.ipynb
df_train_ab = mq_to_ab(df_train)

In [6]:
# Perform Linear Regression
linear_reg = sk_lm.LinearRegression()
linear_reg.fit(y = df_train[cat].values.reshape(-1,1), X = df_train[pred].values.reshape(-1,1))
intercept = linear_reg.intercept_[0]
slope = linear_reg.coef_[0][0]

# Adjusted predictions
df_train[f'{cat}_adj'] = intercept + slope * df_train[pred]

In [ ]:
# Table 4.6
print(intercept)
print(slope)

### Testing

In [8]:
# Sample Test Set, not overlapping with Training Set
df_no_train = df_comp.drop(df_train.index)
# sample for test set
df_test = df_no_train.sample(n = n, axis=0, random_state=42)

In [9]:
# Adjust Test Set
df_test[f'{cat}_adj'] = intercept + slope * df_test[pred]
df_test['adj_diff'] = np.subtract(df_test[cat].values, df_test[f'{cat}_adj'].values)

In [ ]:
# Error for Test Set
print(df_test['adj_diff'].mean())
print(df_test['adj_diff'].std())

### Complete Data Set

In [11]:
# Adjust Data
df_comp[f'{cat}_adj'] = intercept + slope * df_comp[pred]
df_comp['adj_diff'] = np.subtract(df_comp[cat].values, df_comp[f'{cat}_adj'].values)

In [ ]:
# Error after Linear Regression
# Table 4.7
print(df_comp['adj_diff'].mean())
print(df_comp['adj_diff'].std())

In [ ]:
# Delta95 after Linear Regression
# Table 4.8
%run alpha_pept_deep_methods.ipynb
delta95 = percentiles(df_comp, 'adj_diff')
print(f"2.5 Percentile: {delta95[0]}, 97.5 Percentile: {delta95[1]}, Delta95: {delta95[2]}")

# Transfer Learning (Random Training Set)

## Training

In [14]:
# Sample Training Set
n = 10000
df_train = df.sample(n=n, axis=0,random_state=42)
%run alpha_pept_deep_methods.ipynb
df_train_ab = mq_to_ab(df_train)

In [ ]:
# Train Model
models = ModelManager(device = 'gpu')
models.load_installed_models()
models.train_ccs_model(df_train_ab)

## Testing

In [16]:
# Sampling Test Set, no Overlap with Training Set
df_test = df.drop(df_train.index)
df_test = df_test.sample(n = n, axis=0, random_state = 42)

In [ ]:
# Predict
%run alpha_pept_deep_methods.ipynb
df_test_ab = mq_to_ab(df_test)
prediction_test = models.predict_mobility(df_test_ab)
df_comp_test = ab_to_mq(df_test, prediction_test)

In [ ]:
# Error for Test Set
cat = 'ccs' #'IM'
print(df_comp_test[f'{cat}_error'].mean())
print(df_comp_test[f'{cat}_error'].std())

## Complete Data Set

In [ ]:
# predict
%run alpha_pept_deep_methods.ipynb
df_ab = mq_to_ab(df)
prediction = models.predict_mobility(df_ab)
df_comp = ab_to_mq(df, prediction)

In [ ]:
# Error after Transfer Learning
# Table 4.9
cat = 'ccs' #'IM'
print(df_comp[f'{cat}_error'].mean())
print(df_comp[f'{cat}_error'].std())

In [ ]:
# Delta95 after Transfer Learning
# Table 4.10
%run alpha_pept_deep_methods.ipynb
delta95 = percentiles(df_comp, f'{cat}_error')
print(f"2.5 Percentile: {delta95[0]}, 97.5 Percentile: {delta95[1]}, Delta95: {delta95[2]}")

In [ ]:
# Error per Charge after Transfer Learning
# Table 4.11
cat = 'ccs' # 'IM'
for charge in df_comp['Charge'].unique():
    print(df_comp[df_comp['Charge']==charge][f'{cat}_error'].mean())
    print(df_comp[df_comp['Charge']==charge][f'{cat}_error'].std())

In [ ]:
# Error over Biological Replicates 
# Table 4.15
cat = 'ccs' # 'IM'
df_51 = df_comp[df_comp["Experiment"]=='P064051']
print(len(df_51))
print(df_51[f'{cat}_error'].mean())
print(df_51[f'{cat}_error'].std())
df_64 = df_comp[df_comp["Experiment"]=='P064064']
print(len(df_64))
print(df_64[f'{cat}_error'].mean())
print(df_64[f'{cat}_error'].std())
df_28 = df_comp[df_comp["Experiment"]=='P064428']
print(len(df_28))
print(df_28[f'{cat}_error'].mean())
print(df_28[f'{cat}_error'].std())

# Transfer Learning with Charge-balanced Training Set

## Training

In [32]:
# Sample Training Set
# discard Charge 1, because of too few measurements
n = 10000

df_2 = df[df['Charge']==2].sample(n = round(n/3), axis = 0, random_state =  42)
df_3 = df[df['Charge']==3].sample(n = round(n/3), axis = 0, random_state = 42)
df_4 = df[df['Charge']==4].sample(n= round(n/3), axis = 0, random_state = 42)

df_train = pd.concat(objs=[df_2, df_3, df_4])
%run alpha_pept_deep_methods.ipynb
df_train_ab = mq_to_ab(df_train)

In [ ]:
# Train Model
models = ModelManager(device = 'gpu')
models.load_installed_models()
models.train_ccs_model(df_train_ab)

## Complete Data Set

In [ ]:
# Delta95 after Transfer Learning
# Table 4.10
cat = 'ccs' #'IM'
%run alpha_pept_deep_methods.ipynb
delta95 = percentiles(df_comp, f'{cat}_error')
print(f"2.5 Percentile: {delta95[0]}, 97.5 Percentile: {delta95[1]}, Delta95: {delta95[2]}")

In [ ]:
# predict whole dataset
%run alpha_pept_deep_methods.ipynb
df_ab = mq_to_ab(df)
prediction = models.predict_mobility(df_ab)


In [38]:
df_comp = ab_to_mq(df, prediction)

In [ ]:
# Error per Charge after Transfer Learning with charge-balanced Training Set
# Table 4.12
cat = 'ccs' # 'IM'
for charge in df_comp['Charge'].unique():
    print(df_comp[df_comp['Charge']==charge][f'{cat}_error'].mean())
    print(df_comp[df_comp['Charge']==charge][f'{cat}_error'].std())

In [ ]:
# Error after Transfer Learning with charge-balanced Training Set
# Table 4.13
print(df_comp[f'{cat}_error'].mean())
print(df_comp[f'{cat}_error'].std())

In [ ]:
# Delta95 after Transfer Learning with charge-balanced Training Set
# Table 4.14
%run alpha_pept_deep_methods.ipynb
delta95 = percentiles(df_comp, f'{cat}_error')
print(f"2.5 Percentile: {delta95[0]}, 97.5 Percentile: {delta95[1]}, Delta95: {delta95[2]}")

# Transfer Learning (Biological Replicate-specific)

In [43]:
# Split DAta into the 3 Biological Replicates
df_51 = df[df["Experiment"]=='P064051']
df_64 = df[df["Experiment"]=='P064064']
df_28 = df[df["Experiment"]=='P064428']

In [44]:
# Choose Biological Replicate
df_ex = df_51 #P064051
#df_ex = df_64 #P064064
#df_ex = dF_28 #P064428

In [45]:
# Sample Training Set
df_train = df.sample(n = round(0.2*len(df_ex)),axis=0, random_state=42)
%run alpha_pept_deep_methods.ipynb
df_train_ab = mq_to_ab(df_train)

In [ ]:
# Train Model
models = ModelManager(device = 'gpu')
models.load_installed_models()
models.train_ccs_model(df_train_ab)

In [ ]:
# Prediction
%run alpha_pept_deep_methods.ipynb
df_ab = mq_to_ab(df_ex)
prediction = models.predict_mobility(df_ab)
df_comp = ab_to_mq(df_ex, prediction)

In [ ]:
# Error for Biological Replicate
# Table 4.16
cat = 'ccs' #'IM'
print(df_comp[f'{cat}_error'].mean())
print(df_comp[f'{cat}_error'].std())

In [ ]:
# Delta95 for Biological Replicate
# Table 4.16
%run alpha_pept_deep_methods.ipynb
delta95 = percentiles(df_comp, f'{cat}_error')
print(f"2.5 Percentile: {delta95[0]}, 97.5 Percentile: {delta95[1]}, Delta95: {delta95[2]}")

In [ ]:
# Mean for all Biological Replicates
# Table 4.15
%run alpha_pept_deep_methods.ipynb
df_ab = mq_to_ab(df)
prediction = models.predict_mobility(df_ab)
df_comp = ab_to_mq(df, prediction)

for replicate in df_comp['Experiment'].unique():
    df_exp = df_comp[df_comp['Experiment']==replicate]
    print(df_exp[f'{cat}_error'].mean())
    print(df_exp[f'{cat}_error'].std())


# Transfer Learning (Rawfile wise)

In [ ]:
# Perform Transfer Learning per Rawfile
result_dict = {}
for experiment in df['Experiment'].unique():
    print(experiment)
    df_ex = df[df['Experiment']== experiment]
    result_dict[experiment] = {}
    # raw file wise
    for raw_file in df_ex['Raw file']:
        print(raw_file)
        df_raw = df_ex[df_ex['Raw file']== raw_file]
        df_ab = mq_to_ab(df_raw)
        df_train = df_ab.sample(n = round(0.2*len(df_ab)), axis = 0, random_state = 42)
        models = ModelManager(device = 'gpu')
        models.load_installed_models()      
        models.train_ccs_model(df_train)
        prediction = models.predict_mobility(df_ab)
        df_result = ab_to_mq(df_raw, prediction)
        mean = df_result['ccs_error'].mean()
        perc_low = np.percentile(df_result['ccs_error'], 2.5)
        perc_up = np.percentile(df_result['ccs_error'], 97.5)
        window = (perc_low)*(-1)+perc_up
        result_dict[experiment][raw_file] = [mean, perc_low, perc_up, window]      


In [ ]:
# prepare Data for plotting
result_51 = pd.DataFrame.from_dict(result_dict['P064051'])
result_51 = result_51.T
result_51.columns = ['Mean', 'Lower bound', 'Upper bound', 'Window']
result_64 = pd.DataFrame.from_dict(result_dict['P064064'])
result_64 = result_64.T
result_64.columns = ['Mean', 'Lower bound', 'Upper bound', 'Window']
result_28 = pd.DataFrame.from_dict(result_dict['P064428'])
result_28 = result_28.T
result_28.columns = ['Mean', 'Lower bound', 'Upper bound', 'Window']

In [ ]:
# Error Bar Plot: Delta95 per Rawfile
# Figure 4.11
result_dfs = [result_51, result_64, result_28]

for df in result_dfs:
    df['error_lower'] = df['Mean'] - df['Lower bound']
    df['error_upper'] = df['Upper bound'] - df['Mean']

    # Plot setup
    plt.figure(figsize=(15, 6))

    # Plotting the error bars
    plt.errorbar(x=range(len(df)), y=df['Mean'], 
                yerr=[df['error_lower'], df['error_upper']], 
                fmt='o', capsize=5, capthick=2,   elinewidth=2)#ecolor='gray',color='blue',
    plt.scatter(x= range(len(df)), y=df['Mean'],  label='CCS prediction', zorder=5)
    # Customize the plot
    plt.xlabel('Fractions')
    plt.ylabel('Delta 95 bounds')
    plt.grid(False)
    plt.hlines(y = 0.0, xmin = -1.5, xmax= 48.5, linestyles='--', colors='grey')
    plt.show()

In [ ]:
# Histogram: Delta95 per Biological Replicate for every Raw File
# Figure 4.12
result_51['Source'] = 'P064051'
result_64['Source'] = 'P064064'
result_28['Source'] = 'P064428'
df_all = pd.concat([result_51, result_64, result_28],ignore_index=True)
plt.figure(figsize=(10, 6))
sns.histplot(x=df_all['Window'],hue=df_all['Source'], multiple='dodge', palette='viridis', bins = 48)
plt.xlabel('Window size')

## Try different training sizes

In [ ]:
rawfile_sample = ['5471_P064051_R1_U23_GG9_1_2596', '5471_P064051_R1_U38_GF11_1_2611', '5471_P064051_R1_U13_GD8_1_2586']
train_size = [0.20,0.30,0.40,0.50,0.60,0.70,0.80,0.90,1.0]

In [ ]:
# Train CCS Model with different Training Set Sizes
for raw_file in rawfile_sample:
    # set aside training set
    df_raw_file = df[df['Raw file']==raw_file]    
    test_set_mq = df_raw_file.sample(n = round(len(df_raw_file)*0.1), axis=0, random_state=42)
    df_raw_file.drop(test_set_mq.index, inplace=True)
    test_set = mq_to_ab(test_set_mq)
    df_train_ab = mq_to_ab(df_raw_file)
    result_dict = {}

    # Train with every Training Set Size and predict all on the Test Set
    for size in train_size:
        df_train = df_train_ab.sample(n = round((size)*len(df_train_ab)),axis =0,  random_state=42)
        models = ModelManager(device = 'gpu')
        models.load_installed_models()      
        models.train_ccs_model(df_train)
        prediction = models.predict_mobility(test_set)
        df_result = ab_to_mq(test_set_mq, prediction.copy())
        mean = df_result['ccs_error'].mean()
        delta95 = percentile(df_result, 'CCS')
        result_dict[size] = [mean, delta95[0], delta95[1], delta95[2]]  
    with open(f'train_size_eval/51_{raw_file}.pkl', 'wb') as f:
        pickle.dump(result_dict, f)           


In [ ]:
# Prepare caluclated Data for Plotting
with open('train_size_eval/51_5471_P064051_R1_U13_GD8_1_2586.pkl', 'rb') as f:
    result_dict_1 = pickle.load(f)

result_1 = pd.DataFrame.from_dict(result_dict_1)
result_1 = result_1.T
result_1.columns = ['Mean', 'Lower bound', 'Upper bound', 'Window']

with open('train_size_eval/51_5471_P064051_R1_U23_GG9_1_2596.pkl', 'rb') as f:
    result_dict_2 = pickle.load(f)

result_2 = pd.DataFrame.from_dict(result_dict_2)
result_2 = result_2.T
result_2.columns = ['Mean', 'Lower bound', 'Upper bound', 'Window']

with open('train_size_eval/51_5471_P064051_R1_U38_GF11_1_2611.pkl', 'rb') as f:
    result_dict_3 = pickle.load(f)

result_3 = pd.DataFrame.from_dict(result_dict_3)
result_3 = result_3.T
result_3.columns = ['Mean', 'Lower bound', 'Upper bound', 'Window']

result_1['Source']='1_5471_P064051_R1_U13_GD8_1_2586'
result_1['size'] = result_1.index
result_2['Source']='51_5471_P064051_R1_U23_GG9_1_2596'
result_2['size'] = result_2.index
result_3['Source']='51_5471_P064051_R1_U38_GF11_1_2611'
result_3['size'] = result_3.index
df_merged = pd.concat(objs=[result_1, result_2, result_3], axis=0)

In [ ]:
# Scatter Plot: Delta95 per Training Set Size for every Rawfile
# Figure 4.13
g = sns.scatterplot(data=df_merged, x='size', y='Window', hue='Source', palette='viridis')
plt.legend(title = 'Raw file', bbox_to_anchor=(1.05, 1.0), loc='upper left')
plt.xlabel('relative Training Set Size')
plt.ylabel('∆95')
plt.figure(figsize=(16, 8))